### Intro
This part is mostly going to follow Michael's notebook as is, we will replace matplotlib with plotly as I like plotly more. Anyway, let's load the data and get cracking

### Imports and Settings

In [4]:
# Internals

# Externals
from rich import print

# Analytics
import pandas as pd
from sklearn.datasets import fetch_openml

# Project specific

pd.options.plotting.backend = "plotly"


##### Load dataset

In [3]:
dataset = fetch_openml(data_id=42803, as_frame=True)
df_X = dataset["frame"]
df_X


,Accident_Index,Vehicle_Reference_df_res,Vehicle_Type,Towing_and_Articulation,Vehicle_Manoeuvre,Vehicle_Location-Restricted_Lane,Junction_Location,Skidding_and_Overturning,Hit_Object_in_Carriageway,Vehicle_Leaving_Carriageway,...,Age_Band_of_Casualty,Casualty_Severity,Pedestrian_Location,Pedestrian_Movement,Car_Passenger,Bus_or_Coach_Passenger,Pedestrian_Road_Maintenance_Worker,Casualty_Type,Casualty_Home_Area_Type,Casualty_IMD_Decile
0,201501BS70001,1.0,19.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,...,7.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,NaN,NaN
1,201501BS70002,1.0,9.0,0.0,9.0,0.0,8.0,0.0,0.0,0.0,...,5.0,3.0,9.0,9.0,0.0,0.0,2.0,0.0,1.0,3.0
2,201501BS70004,1.0,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,...,6.0,3.0,1.0,3.0,0.0,0.0,2.0,0.0,1.0,6.0
3,201501BS70005,1.0,9.0,0.0,9.0,0.0,2.0,0.0,0.0,0.0,...,2.0,3.0,5.0,1.0,0.0,0.0,2.0,0.0,1.0,2.0
4,201501BS70008,1.0,1.0,0.0,18.0,0.0,8.0,0.0,0.0,0.0,...,8.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363238,2015984141415,13.0,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,1.0,3.0,0.0,0.0,2.0,0.0,0.0,9.0,1.0,NaN
363239,2015984141415,13.0,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,5.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,1.0,2.0
363240,2015984141415,13.0,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,4.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,5.0
363241,2015984141415,13.0,9.0,0.0,18.0,0.0,0.0,0.0,0.0,5.0,...,6.0,3.0,0.0,0.0,0.0,0.0,0.0,9.0,3.0,NaN


## Structural investigation

In [8]:
# Dataset shape
print(df_X.shape)


(363243, 67)

In [9]:
print(pd.value_counts(df_X.dtypes))


float64    61
object      6
dtype: int64

#### 1.1 Structure of non-numerical features

In [12]:
df_X.select_dtypes(exclude="number").head()


,Accident_Index,Sex_of_Driver,Date,Time,Local_Authority_(Highway),LSOA_of_Accident_Location
0,201501BS70001,1.0,12/01/2015,18:45,E09000020,E01002825
1,201501BS70002,1.0,12/01/2015,07:50,E09000020,E01002820
2,201501BS70004,1.0,12/01/2015,18:08,E09000020,E01002833
3,201501BS70005,1.0,13/01/2015,07:40,E09000020,E01002874
4,201501BS70008,1.0,09/01/2015,07:30,E09000020,E01002814


In [13]:
# Sex_of_Driver is stored of non-numerical feature for unknown reasons so let's fix that
df_X["Sex_of_Driver"] = df_X["Sex_of_Driver"].astype("float")


In [14]:
df_X.describe(exclude="number")


,Accident_Index,Date,Time,Local_Authority_(Highway),LSOA_of_Accident_Location
count,363243,319866,319822,319866,298758
unique,140056,365,1439,204,25979
top,201543P296025,14/02/2015,17:30,E10000017,E01028497
freq,1332,2144,2972,8457,1456


#### 1.2 Structure of numerical features

In [19]:
unique_values = df_X.select_dtypes(include="number").nunique().sort_values()
unique_values.plot.bar(title="Unique values per numerical feature", log_y=True)
